In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/path/to/MNIST_data/",one_hot=True)

Extracting /path/to/MNIST_data/train-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /path/to/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
mnist.train.num_examples

55000

In [5]:
mnist.test.num_examples

10000

In [6]:
mnist.validation.num_examples

5000

In [8]:
mnist.train.images[0]
mnist.train.labels[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.])

In [9]:
batch_size=100
xs,ys=mnist.train.next_batch(batch_size)


In [10]:
xs.shape

(100, 784)

In [11]:
ys.shape

(100, 10)

In [7]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#输入层和输出层的节点数
INPUT_NODE=784
OUTPUT_NODE=10

#配置神经网络的参数
LAYER1_NNODE=500

BATCH_SIZE=100
LEARNING_RATE_BASE=0.8   #基础的学习率
LEARNING_RATE_DECAY=0.99   #学习率的衰减率
REGULARIZATION_RATE=0.0001    #描述模型复杂度的正则化项在损失函数中的系数
TRAINING_STEPS=5000          #循环轮次
MOVING_AVERAGE_DECAY=0.99        #滑动平均衰减率

def inference(input_tensor,avg_class,w1,b1,w2,b2):
    #当没有提供滑动平滑类时，直接使用参数挡墙的取值
    if(avg_class==None):
        layer1=tf.nn.relu(tf.matmul(input_tensor,w1)+b1)
        return tf.matmul(layer1,w2)+b2
    else:
        #首先使用avg_class.average函数来计算得出变量的滑动平均值。
        #然后再计算相应的神经网络的前向传播结果。
        layer1=tf.nn.relu(tf.matmul(input_tensor,avg_class.average(w1))+avg_class.average(b1))
        return tf.matmul(layer1,avg_class.average(w2))+avg_class.average(b2)

#训练模型的过程
def train(mnist):
    x=tf.placeholder(tf.float32,shape=(None,INPUT_NODE),name="X_input")
    y_=tf.placeholder(tf.float32,shape=(None,OUTPUT_NODE),name="Y_input")
    
    #生成隐藏层参数
    w1=tf.Variable(tf.random_normal([INPUT_NODE,LAYER1_NNODE],stddev=0.1))
    b1=tf.Variable(tf.constant(0.1,shape=[LAYER1_NNODE]))
    
    w2=tf.Variable(tf.random_normal([LAYER1_NNODE,OUTPUT_NODE],stddev=0.1))
    b2=tf.Variable(tf.constant(0.1,shape=[OUTPUT_NODE]))
    #计算当前参数下的神经网络的前向传播结果，这里给出用于计算滑动平均的类为None
    #所以函数不会使用参数的滑动平均值
    y=inference(x,None,w1,b1,w2,b2)
    #定义存储训练轮次的变量，改变量为不可训练变量。
    global_step=tf.Variable(0,trainable=False)
    
    #给定滑动平均衰减率和训练轮数变量，初始化滑动平均类,给定训练轮次的变量可以加快训练早期变量的更新速度
    variable_averages=tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    #在所有代表神经网络的参数上使用滑动平均，其他辅助变量就不用了
    variable_averages_op=variable_averages.apply(tf.trainable_variables())
    
    #计算使用了滑动平均之后的前向传播结果
    average_y=inference(x,variable_averages,w1,b1,w2,b2)
    
    #计算交叉熵用于刻画预测值和真实值之间差距的损失函数。
    cross_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    #计算当前batch中所有样例交叉熵的平均值
    cross_entropy_mean=tf.reduce_mean(cross_entropy)
    
    #计算L2正则化损失函数
    regularizer=tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    #计算模型的正则化损失，一般只计算神经网络边上权重的正则化损失，而不使用偏置项。
    regularization=regularizer(w1)+regularizer(w2)
    #总损失等于交叉熵损失加上正则化损失
    loss=cross_entropy_mean+regularization
    #设置指数衰减学习率
    learning_rate=tf.train.exponential_decay(LEARNING_RATE_BASE,global_step,mnist.train.num_examples/BATCH_SIZE,LEARNING_RATE_DECAY)
    #这里使用梯度下降算法来优化损失函数
    train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)

    #在训练神经网络模型时候，过一变数据既要通过反向传播来更新神经网络中的参数，又要更新每一个参数的滑动平均值
    with tf.control_dependencies([train_step,variable_averages_op]):
        train_op=tf.no_op(name='train')
    #检验使用了滑动平均模型的神经网络前向传播结果是否正确
    correct_prediction=tf.equal(tf.argmax(average_y,1),tf.argmax(y_,1))
    #正确率
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

    #初始化会话并开始训练过程
    with tf.Session() as sess:
        tf.initialize_all_variables().run()
        #准备验证数据
        validate_feed={x:mnist.validation.images,y_:mnist.validation.labels}
        #准备测试数据
        test_feed={x:mnist.test.images,y_:mnist.test.labels}
        #迭代的训练神经网络
        for i in range(TRAINING_STEPS):
            #没1千轮输出一次在验证集上的测试结果
            if(i%1000==0):
                validate_acc=sess.run(accuracy,feed_dict=validate_feed)
                print("After %d training  step(S),validation accuracy using average model is %g " % (i,validate_acc))
            #产生这一轮使用的一个batch的训练数据，并运行训练过程
            xs,ys=mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op,feed_dict={x:xs,y_:ys})
        #在训练结束之后，在测试数据上检验模型的最终准确率
        test_acc=sess.run(accuracy,feed_dict=test_feed)
        print("After %d training  step(S),test accuracy using average model is %g " % (i,test_acc))

def main(argv=None):
    mnist=input_data.read_data_sets("/path/to/MNIST_data/",one_hot=True)
    train(mnist)
if __name__=='__main__':
    tf.app.run()

Extracting /path/to/MNIST_data/train-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /path/to/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
After 0 training  step(S),validation accuracy using average model is 0.0854 
After 1000 training  step(S),validation accuracy using average model is 0.9784 
After 2000 training  step(S),validation accuracy using average model is 0.9816 
After 3000 training  step(S),validation accuracy using average model is 0.9832 
After 4000 training  step(S),validation accuracy using average model is 0.9834 
After 4999 training  step(S),test accuracy using average model is 0.982 


SystemExit: 

C:\Users\pujing\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [44]:
with tf.variable_scope("foo"):
    v=tf.get_variable("v",shape=[1],initializer=tf.constant_initializer(1.0))

In [46]:
with tf.variable_scope("foo",reuse=True):
    v=tf.get_variable("v",shape=[1])

In [14]:
import tensorflow as tf
v1=tf.Variable(tf.constant(1.0,shape=[1]),name="v1")
v2=tf.Variable(tf.constant(2.0,shape=[1]),name="v2")
result=v1+v2

saver=tf.train.Saver()
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    saver.save(sess,"/path/Tensorflow/model.ckpt")

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [16]:
#加载模型
import tensorflow as tf
v1=tf.Variable(tf.constant(1.0,shape=[1]),name="v1")
v2=tf.Variable(tf.constant(2.0,shape=[1]),name="v2")
result=v1+v2

saver=tf.train.Saver()
with tf.Session() as sess:
    #tf.initialize_all_variables().run()
    saver.restore(sess,"/path/Tensorflow/model.ckpt.meta")
    print(sess.run(result))

INFO:tensorflow:Restoring parameters from /path/Tensorflow/model.ckpt.meta


DataLossError: Unable to open table file \path\Tensorflow\model.ckpt.meta: Data loss: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?
	 [[Node: save_9/RestoreV2 = RestoreV2[dtypes=[DT_INT32], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save_9/Const_0, save_9/RestoreV2/tensor_names, save_9/RestoreV2/shape_and_slices)]]

Caused by op 'save_9/RestoreV2', defined at:
  File "C:\Users\pujing\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\pujing\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\pujing\Anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\pujing\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\pujing\Anaconda3\lib\site-packages\ipykernel\kernelapp.py

In [5]:
import tensorflow as tf
from tensorflow.python.framework import graph_util
v1=tf.Variable(tf.constant(1.0,shape=[1]),name="v1")
v2=tf.Variable(tf.constant(2.0,shape=[1]),name="v2")
result=v1+v2
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    #导出当前计算图的GraphDef部分，只需要这一部分就可以完成输入到输出的计算过程。
    graph_def=tf.get_default_graph().as_graph_def()
    output_graph_def=graph_util.convert_variables_to_constants(sess,graph_def,['add'])
    
    #将导出的模型写入文件。
    with tf.gfile.GFile("/path/to/model/combined_model.pb","wb") as f:
        f.write(output_graph_def.SerializeToString())

Instructions for updating:
Use `tf.global_variables_initializer` instead.
INFO:tensorflow:Froze 2 variables.
Converted 2 variables to const ops.


In [8]:
import tensorflow as tf
from tensorflow.python.platform import gfile
with tf.Session() as sess:
    model_file="/path/to/model/combined_model.pb"
    #读取保存的模型文件
    with gfile.FastGFile(model_file,'rb') as f:
        graph_def=tf.GraphDef()
        graph_def.ParseFromString(f.read())
    result=tf.import_graph_def(graph_def,return_elements=["add:0"])
    print(sess.run(result))

[array([ 3.], dtype=float32)]


In [1]:
import tensorflow as tf
#定义神经网络的相关参数
INPUT_NODE=784
OUTPUT_NODE=10
LAYER1_NODE=500

def get_weigth_variable(shape,regularizer):
    #weigths=tf.Variable()
    weigths = tf.get_variable("weights", shape, initializer=tf.truncated_normal_initializer(stddev=0.1))
    #当给出了正则化生成函数时，将当前变量的正则化损失加入名字为losses的集合中去，
    if(regularizer!=None):
        tf.add_to_collection('losses',regularizer(weigths))
    return weigths

#定义神经网络的前向传播过程
def inference(input_tensor,regularizer):
    #声明第一层神经网络的变量并完成前向传播过程
    with tf.variable_scope('layer1',reuse=None):
        weigths=get_weigth_variable([INPUT_NODE,LAYER1_NODE],regularizer)
        biases=tf.get_variable("biases",[LAYER1_NODE],initializer=tf.constant_initializer(0.0))
        layer1=tf.nn.relu(tf.matmul(input_tensor,weigths)+biases)
    #声明第二层神经网络的变量并完成前向传播过程
    with tf.variable_scope('layer2',reuse=None):
        weigths=get_weigth_variable([LAYER1_NODE,OUTPUT_NODE],regularizer)
        biases=tf.get_variable("biases",[OUTPUT_NODE],initializer=tf.constant_initializer(0.0))
        layer2=tf.matmul(layer1,weigths)+biases
    return layer2

In [2]:
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#配置神经网络参数
BATCH_SIZE=100
LEARNING_RATE_BASE=0.8   #基础学习率
LEARNING_RATE_DECAY=0.99     #学习率的衰减率
REGULARAZTION_RATE=0.0001  #描述模型复杂度的正则化项在损失函数中的系数
TRAINING_STEPS=5000       #训练次数
MOVING_AVERAGE_DECAY=0.99     #滑动平均衰减率

#模型保存的路径和文件名
MODEL_SAVE_PATH="/path/to/model/"
MODEL_NAME="model.ckpt"


def train(mnist):
    x=tf.placeholder(tf.float32,[None,INPUT_NODE],name="x")
    y_=tf.placeholder(tf.float32,[None,OUTPUT_NODE],name="y")
    
    #L2正则化损失函数
    regularizer=tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)
    #直接使用前向传播算法
    y=inference(x,regularizer)
    #定义训练轮次的变量
    global_step=tf.Variable(0,trainable=False)
    
    #接下来定义损失函数，学习率，滑动平均值，以及训练过程
    variable_averages=tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    variable_averages_op=variable_averages.apply(tf.trainable_variables())
    
    #交叉熵
    cross_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    cross_entropy_mean=tf.reduce_mean(cross_entropy)
    
    #总损失
    loss=cross_entropy_mean+tf.add_n(tf.get_collection('losses'))
    
    #设置学习率和优化损失函数
    learning_rate=tf.train.exponential_decay(LEARNING_RATE_BASE,global_step,mnist.train.num_examples/BATCH_SIZE,LEARNING_RATE_DECAY)
    train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step)
    with tf.control_dependencies([train_step,variable_averages_op]):
        train_op=tf.no_op(name='train')
        
    #初始化持久化类
    saver=tf.train.Saver()
    with tf.Session() as sess:
        tf.initialize_all_variables().run()
        for i in range(TRAINING_STEPS):
            xs,ys=mnist.train.next_batch(BATCH_SIZE)
            _,loss_value,step=sess.run([train_op,loss,global_step],feed_dict={x:xs,y_:ys})
            
            #每1000轮保存一次模型
            if(i%1000==0):
                print("After %d training step(s),loss on training batch is %g." %(step,loss_value))
                saver.save(sess,os.path.join(MODEL_SAVE_PATH,MODEL_NAME),global_step=global_step)

def main(argv=None):
    mnist=input_data.read_data_sets("/path/to/MNIST_data/",one_hot=True)
    train(mnist)
if __name__=='__main__':
    tf.app.run()
    

Extracting /path/to/MNIST_data/train-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /path/to/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
After 1 training step(s),loss on training batch is 3.19616.
After 1001 training step(s),loss on training batch is 0.278093.
After 2001 training step(s),loss on training batch is 0.152246.
After 3001 training step(s),loss on training batch is 0.128788.
After 4001 training step(s),loss on training batch is 0.114241.


SystemExit: 

C:\Users\pujing\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
#每10秒加载一次新的模型
import time
EVAL_INTERVAL_SECS=10
def evaluate(mnist):
    with tf.Graph().as_default() as g:
        #定义输入输出的格式
        x=tf.placeholder(tf.float32,[None,INPUT_NODE],name='X')
        y_=tf.placeholder(tf.float32,[None,OUTPUT_NODE],name='Y')
        validate_feed={x:mnist.validation.images,y_:mnist.validation.labels}
        #直接调用封装好的函数计算前向传播结果，因为测试的时候不需要关系正则化损失的值，所以这里用于计算正则化损失的函数设置为None
        y=inference(x,None)
        
        #使用前向传播的结果计算准确率，如果需要对未知的样例进行分类，，俺么使用tf.argmax(y,1)就可以得到输出样例的预测类别了。
        correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
        accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        #通过变量重命名的方式来加载模型，这样在前向传播过程中，就不需要调用滑动平均的函数来获取平均值，这样就可以完全公用前面的前向传播过程。
        variable_averages=tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        variables_to_restore=variable_averages.variables_to_restore()
        saver=tf.train.Saver(variables_to_restore)
        while True:
            with tf.Session() as sess:
                ckpt=tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
                if(ckpt and ckpt.model_checkpoint_path):
                    #加载模型
                    saver.restore(sess,ckpt.model_checkpoint_path)
                    #通过文件名保存模型保存时迭代的轮次
                    global_step=ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    accuracy_score=sess.run(accuracy,feed_dict=validate_feed)
                    print("After %s training step(s),validation accuracy=%g " % (global_step,accuracy_score))
                else:
                    print("No ckickpoint file found")
                    return
                time.sleep(EVAL_INTERVAL_SECS)
def main(argv=None):
    mnist=input_data.read_data_sets("/path/to/MNIST_data/",one_hot=True)
    evaluate(mnist)
if __name__=='__main__':
    tf.app.run()

Extracting /path/to/MNIST_data/train-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /path/to/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Restoring parameters from /path/to/model/model.ckpt-4001
After 4001 training step(s),validation accuracy=0.9832 
INFO:tensorflow:Restoring parameters from /path/to/model/model.ckpt-4001
After 4001 training step(s),validation accuracy=0.9832 
INFO:tensorflow:Restoring parameters from /path/to/model/model.ckpt-4001
After 4001 training step(s),validation accuracy=0.9832 
INFO:tensorflow:Restoring parameters from /path/to/model/model.ckpt-4001
After 4001 training step(s),validation accuracy=0.9832 
INFO:tensorflow:Restoring parameters from /path/to/model/model.ckpt-4001
After 4001 training step(s),validation accuracy=0.9832 
INFO:tensorflow:Restoring parameters from /path/to/model/model.ckpt-4001
After 4001 training step(s),validation accuracy=0.98

KeyboardInterrupt: 